In [1]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import math
import warnings
warnings.filterwarnings("ignore")
# I have used my google drive to load the training and testing data
# Upload the dataset file in your google drive and change the path to run the below line
Labeled_data= pd.read_csv('/content/drive/MyDrive/ML/labeled.csv',dtype='object')
Unlabeled_data= pd.read_csv('/content/drive/MyDrive/ML/Unlabeled.csv',dtype='object')

Mounted at /content/drive


In [2]:
for i in range(len(Labeled_data.iloc[:,-1])):
    if Labeled_data.iloc[:,-1][i] == ' Plastic ':
       Labeled_data.iloc[:,-1][i] = 0
    elif Labeled_data.iloc[:,-1][i] == ' Metal ':
        Labeled_data.iloc[:,-1][i] = 1
    elif Labeled_data.iloc[:,-1][i] == ' Ceramic ':
        Labeled_data.iloc[:,-1][i] = 2

In [3]:
for i in range(len(Unlabeled_data.iloc[:,-1])):
    if Unlabeled_data.iloc[:,-1][i] == ' Plastic ':
       Unlabeled_data.iloc[:,-1][i] = 0
    elif Unlabeled_data.iloc[:,-1][i] == ' Metal ':
        Unlabeled_data.iloc[:,-1][i] = 1
    elif Unlabeled_data.iloc[:,-1][i] == ' Ceramic ':
        Unlabeled_data.iloc[:,-1][i] = 2

In [4]:
unlabeled_data= Unlabeled_data.iloc[:,:].values
unlabeled_data= unlabeled_data.astype('float64')

labeled_data= Labeled_data.iloc[:,:].values
labeled_data= labeled_data.astype('float64')

In [5]:
unlabeled_data.shape

(90, 5)

In [6]:
labeled_data.shape

(30, 5)

In [7]:
# the cartesian distance is been calculated in this function
def cartesian_function(testdata, datas):
    test1=testdata.astype('float64')
    data1=datas.astype('float64')
    diff1 = np.power(test1 - data1, 2)
    difference = np.sum(diff1,axis=1)
    final_diff = np.power(difference, 0.5)
    return final_diff

In [8]:
# The classification of the correct labels are been done in this function
# for k=3 and 5 --> if the knn classifer doesnt come with the majority decision , i have classified as it based on k =1.
def classify(k, label):
    #print(k)
    #print(label)
    output = label[:k]
    m = np.count_nonzero(output == 0)
    #print(m)
    p = np.count_nonzero(output == 1)
    #print(p)
    c = np.count_nonzero(output == 2)
    #print(c)
    prob=[]
    prob.append(m/k)
    prob.append(p/k)
    prob.append(c/k)
    #print(prob)
    if m > p and m > c:
        return 0, prob
    elif p > m and p> c:
        return 1, prob
    elif c > m and c > p:
        return 2, prob
    else:
      output = label[:1].astype(int)
      if(output==1):
        return 1, prob
      elif(output == 2):
        return 2, prob
      elif (output == 0):
        return 0, prob

In [9]:
# this function defines the knn classification by finding the distance and sorting it. 
# this functions calls classify function to classify labels
def knn(testsample, k, data,y_train):
    labels = y_train
    distance = cartesian_function(testsample, data)
    #print(distance)
    final_distance = np.vstack((distance, labels))
    final = final_distance.T[:, 0].argsort()
    final_sorted_distance = final_distance.T[final]
    #print(final_sorted_distance)
    final_sorted_labels = final_sorted_distance.T[1]
    #print(final_sorted_labels)
    return classify(k,final_sorted_labels)

In [10]:
predicted_labels =[]
probabilities=[]
predictions =0
k=3
labeled_features = labeled_data[:,:-1]
labeled_labels = labeled_data[:, -1]
#print(labeled_data.shape)
for i in range(len(unlabeled_data)):
      pred, prob=knn(unlabeled_data[:, :-1][i], k, labeled_features,labeled_labels)
      #print(pred)
      predicted_labels.append(pred)
      probabilities.append(prob)
      #print(pred)
      #print(unlabeled_data[:, -1][i])
      if(pred == unlabeled_data[:, -1][i].astype(int)):
        predictions +=1
print(f"{predictions}/{unlabeled_data.shape[0]} correct predictions ")
print(f'performance : ', (predictions/unlabeled_data.shape[0]) * 100)

36/90 correct predictions 
performance :  40.0


In [11]:
# Define the number of iterations and the number of data items to add in each iteration
num_iterations = 15
batch_size = 1

# Split the labeled data into features and labels
labeled_features = labeled_data[:, :-1]
#print(labeled_features)
labeled_labels = labeled_data[:, -1]
labeled_labels= labeled_labels.astype(int)
k=3

for j in range(num_iterations):
    predicted_labels =[]
    probabilities=[]
    # Train a K-nearest neighbor classifier on the labeled data
    # Use the classifier to make predictions on the unlabeled data
    for i in range(len(unlabeled_data)):
      #print(unlabeled_data[:, :-1][i])
      pred, prob=knn(unlabeled_data[:, :-1][i], k, labeled_features,labeled_labels)
      predicted_labels.append(pred)
      probabilities.append(prob)
    #print(predicted_labels)
    
    # Compute the purity of the prediction for each data point
    max_probabilities = np.max(probabilities, axis=1)
    purity = max_probabilities / np.sum(probabilities, axis=1)
    
    # Select the subset of data with the highest level of certainty
    indices = np.argsort(purity)[-batch_size:]
    selected_data = unlabeled_data[indices]
    selected_labels = np.array(predicted_labels)[indices.astype(int)]

    # Add the selected subset to the labeled dataset and remove them from the unlabeled dataset
    labeled_data = np.vstack([labeled_data, np.hstack([selected_data[:,:-1], selected_labels.reshape(-1, 1)])])
    unlabeled_data = np.delete(unlabeled_data, indices, axis=0)
    
    # Train a new classifier on the expanded labeled dataset
    labeled_features = labeled_data[:, :-1]
    labeled_labels = labeled_data[:, -1]

    predictions =0
    predicted_labels =[]
    probabilities=[]
    for i in range(len(labeled_features)):
      pred, prob=knn(labeled_features[i], k, labeled_features,labeled_labels)
      #print(pred)
      predicted_labels.append(pred)
      probabilities.append(prob)
      if(pred == labeled_labels[i].astype(int)):
        predictions +=1
    print("For Iteration : ",j+1)
    print(f"{predictions}/{labeled_features.shape[0]} correct predictions ")
    print(f'performance : ', (predictions/labeled_features.shape[0]) * 100)


For Iteration :  1
22/31 correct predictions 
performance :  70.96774193548387
For Iteration :  2
23/32 correct predictions 
performance :  71.875
For Iteration :  3
24/33 correct predictions 
performance :  72.72727272727273
For Iteration :  4
25/34 correct predictions 
performance :  73.52941176470588
For Iteration :  5
26/35 correct predictions 
performance :  74.28571428571429
For Iteration :  6
27/36 correct predictions 
performance :  75.0
For Iteration :  7
29/37 correct predictions 
performance :  78.37837837837837
For Iteration :  8
30/38 correct predictions 
performance :  78.94736842105263
For Iteration :  9
31/39 correct predictions 
performance :  79.48717948717949
For Iteration :  10
32/40 correct predictions 
performance :  80.0
For Iteration :  11
33/41 correct predictions 
performance :  80.48780487804879
For Iteration :  12
34/42 correct predictions 
performance :  80.95238095238095
For Iteration :  13
34/43 correct predictions 
performance :  79.06976744186046
For It

In [12]:
unlabeled_data= Unlabeled_data.iloc[:,:].values
unlabeled_data= unlabeled_data.astype('float64')
labeled_data= Labeled_data.iloc[:,:].values
labeled_data= labeled_data.astype('float64')

In [13]:
# Define the number of iterations and the number of data items to add in each iteration
num_iterations = 1
batch_size = 90

# Split the labeled data into features and labels
labeled_features = labeled_data[:, :-1]
#print(labeled_features)
labeled_labels = labeled_data[:, -1]
labeled_labels= labeled_labels.astype(int)
k=3


for j in range(num_iterations):
    predicted_labels =[]
    probabilities=[]
    # Train a K-nearest neighbor classifier on the labeled data
    # Use the classifier to make predictions on the unlabeled data
    for i in range(len(unlabeled_data)):
      #print(unlabeled_data[:, :-1][i])
      pred, prob=knn(unlabeled_data[:, :-1][i], k, labeled_features,labeled_labels)
      predicted_labels.append(pred)
      probabilities.append(prob)
    #print(predicted_labels)
    
    # Compute the purity of the prediction for each data point
    max_probabilities = np.max(probabilities, axis=1)
    purity = max_probabilities / np.sum(probabilities, axis=1)
    
    # Select the subset of data with the highest level of certainty
    indices = np.argsort(purity)[-batch_size:]
    selected_data = unlabeled_data[indices]
    selected_labels = np.array(predicted_labels)[indices.astype(int)]

    # Add the selected subset to the labeled dataset and remove them from the unlabeled dataset
    labeled_data = np.vstack([labeled_data, np.hstack([selected_data[:,:-1], selected_labels.reshape(-1, 1)])])
    unlabeled_data = np.delete(unlabeled_data, indices, axis=0)
    
    # Train a new classifier on the expanded labeled dataset
    labeled_features = labeled_data[:, :-1]
    labeled_labels = labeled_data[:, -1]

    predictions =0
    predicted_labels =[]
    probabilities=[]
    for i in range(len(labeled_features)):
      pred, prob=knn(labeled_features[i], k, labeled_features,labeled_labels)
      #print(pred)
      predicted_labels.append(pred)
      probabilities.append(prob)
      if(pred == labeled_labels[i].astype(int)):
        predictions +=1
    print("For Iteration : ",j+1)
    print(f"{predictions}/{labeled_features.shape[0]} correct predictions ")
    print(f'performance : ', (predictions/labeled_features.shape[0]) * 100)


For Iteration :  1
109/120 correct predictions 
performance :  90.83333333333333


In [14]:
unlabeled_data= Unlabeled_data.iloc[:,:].values
unlabeled_data= unlabeled_data.astype('float64')
labeled_data= Labeled_data.iloc[:,:].values
labeled_data= labeled_data.astype('float64')

In [15]:
# Define the number of iterations and the number of data items to add in each iteration
num_iterations = 18
batch_size = 5

# Split the labeled data into features and labels
labeled_features = labeled_data[:, :-1]
#print(labeled_features)
labeled_labels = labeled_data[:, -1]
labeled_labels= labeled_labels.astype(int)
k=3


for j in range(num_iterations):
    predicted_labels =[]
    probabilities=[]
    # Train a K-nearest neighbor classifier on the labeled data
    # Use the classifier to make predictions on the unlabeled data
    for i in range(len(unlabeled_data)):
      #print(unlabeled_data[:, :-1][i])
      pred, prob=knn(unlabeled_data[:, :-1][i], k, labeled_features,labeled_labels)
      predicted_labels.append(pred)
      probabilities.append(prob)
    #print(predicted_labels)
    
    # Compute the purity of the prediction for each data point
    max_probabilities = np.max(probabilities, axis=1)
    purity = max_probabilities / np.sum(probabilities, axis=1)
    
    # Select the subset of data with the highest level of certainty
    indices = np.argsort(purity)[-batch_size:]
    selected_data = unlabeled_data[indices]
    selected_labels = np.array(predicted_labels)[indices.astype(int)]

    # Add the selected subset to the labeled dataset and remove them from the unlabeled dataset
    labeled_data = np.vstack([labeled_data, np.hstack([selected_data[:,:-1], selected_labels.reshape(-1, 1)])])
    unlabeled_data = np.delete(unlabeled_data, indices, axis=0)
    
    # Train a new classifier on the expanded labeled dataset
    labeled_features = labeled_data[:, :-1]
    labeled_labels = labeled_data[:, -1]

    predictions =0
    predicted_labels =[]
    probabilities=[]
    for i in range(len(labeled_features)):
      pred, prob=knn(labeled_features[i], k, labeled_features,labeled_labels)
      #print(pred)
      predicted_labels.append(pred)
      probabilities.append(prob)
      if(pred == labeled_labels[i].astype(int)):
        predictions +=1
    print("For Iteration : ",j+1)
    print(f"{predictions}/{labeled_features.shape[0]} correct predictions ")
    print(f'performance : ', (predictions/labeled_features.shape[0]) * 100)


For Iteration :  1
26/35 correct predictions 
performance :  74.28571428571429
For Iteration :  2
31/40 correct predictions 
performance :  77.5
For Iteration :  3
36/45 correct predictions 
performance :  80.0
For Iteration :  4
41/50 correct predictions 
performance :  82.0
For Iteration :  5
45/55 correct predictions 
performance :  81.81818181818183
For Iteration :  6
50/60 correct predictions 
performance :  83.33333333333334
For Iteration :  7
54/65 correct predictions 
performance :  83.07692307692308
For Iteration :  8
61/70 correct predictions 
performance :  87.14285714285714
For Iteration :  9
66/75 correct predictions 
performance :  88.0
For Iteration :  10
71/80 correct predictions 
performance :  88.75
For Iteration :  11
76/85 correct predictions 
performance :  89.41176470588236
For Iteration :  12
79/90 correct predictions 
performance :  87.77777777777777
For Iteration :  13
84/95 correct predictions 
performance :  88.42105263157895
For Iteration :  14
88/100 correc

In [16]:
unlabeled_data= Unlabeled_data.iloc[:,:].values
unlabeled_data= unlabeled_data.astype('float64')
labeled_data= Labeled_data.iloc[:,:].values
labeled_data= labeled_data.astype('float64')

In [17]:
# Define the number of iterations and the number of data items to add in each iteration
num_iterations = 5
batch_size = 18

# Split the labeled data into features and labels
labeled_features = labeled_data[:, :-1]
#print(labeled_features)
labeled_labels = labeled_data[:, -1]
labeled_labels= labeled_labels.astype(int)
k=3


for j in range(num_iterations):
    predicted_labels =[]
    probabilities=[]
    # Train a K-nearest neighbor classifier on the labeled data
    # Use the classifier to make predictions on the unlabeled data
    for i in range(len(unlabeled_data)):
      #print(unlabeled_data[:, :-1][i])
      pred, prob=knn(unlabeled_data[:, :-1][i], k, labeled_features,labeled_labels)
      predicted_labels.append(pred)
      probabilities.append(prob)
    #print(predicted_labels)
    
    # Compute the purity of the prediction for each data point
    max_probabilities = np.max(probabilities, axis=1)
    purity = max_probabilities / np.sum(probabilities, axis=1)
    
    # Select the subset of data with the highest level of certainty
    indices = np.argsort(purity)[-batch_size:]
    selected_data = unlabeled_data[indices]
    selected_labels = np.array(predicted_labels)[indices.astype(int)]

    # Add the selected subset to the labeled dataset and remove them from the unlabeled dataset
    labeled_data = np.vstack([labeled_data, np.hstack([selected_data[:,:-1], selected_labels.reshape(-1, 1)])])
    unlabeled_data = np.delete(unlabeled_data, indices, axis=0)
    
    # Train a new classifier on the expanded labeled dataset
    labeled_features = labeled_data[:, :-1]
    labeled_labels = labeled_data[:, -1]

    predictions =0
    predicted_labels =[]
    probabilities=[]
    for i in range(len(labeled_features)):
      pred, prob=knn(labeled_features[i], k, labeled_features,labeled_labels)
      #print(pred)
      predicted_labels.append(pred)
      probabilities.append(prob)
      if(pred == labeled_labels[i].astype(int)):
        predictions +=1
    print("For Iteration : ",j+1)
    print(f"{predictions}/{labeled_features.shape[0]} correct predictions ")
    print(f'performance : ', (predictions/labeled_features.shape[0]) * 100)


For Iteration :  1
41/48 correct predictions 
performance :  85.41666666666666
For Iteration :  2
58/66 correct predictions 
performance :  87.87878787878788
For Iteration :  3
74/84 correct predictions 
performance :  88.09523809523809
For Iteration :  4
92/102 correct predictions 
performance :  90.19607843137256
For Iteration :  5
105/120 correct predictions 
performance :  87.5


The accuracy of the data is high when we pass the test data to the model and predict and add that test data as training data. doing so we are gradually incresing the traindata.

Self training method is vulnerabe to outliers. since we take a 'k' neighbors much small with in the dataset, and first 20 training data couls be biased and model starts to predict according to initial selection training data.

The result is bad when we pass all the test data at once and slightly better when we pass then batch wise and add the predicted to train.

since self training is so vulnerable co-training is another method to overcome this issue.